In [1]:
""" data processing script """

' data processing script '

In [2]:
# import modules
import os
import re
import pandas as pd


#### Naming Conventions Methods

In [3]:
def name_alpha_numeric_ended(file_name:str):
    """ end name with alphanumeric character only """
    if file_name[-1].isalnum():
        return file_name
    else:
        file_name=file_name[:-1]
    return name_alpha_numeric_ended(file_name)

In [4]:
def revamp_file_name(file_name:str):
    """ rename csv file according to convention """
    # lower name, 
    # replace space and dash with underscore,
    # remove comma, colon, semicolon etc from name,
    # last character should be alphabat,
    file_name= file_name.lower().translate(str.maketrans({' ':'_','-':'_','"':'',"'":""}))
    name, ext = os.path.splitext(file_name)
    name= name_alpha_numeric_ended(name)
    file_name=name+ext
    file_name= file_name.replace('mens','men')
    return file_name

##### Populate Gender Column

In [5]:
# append gender there
boys_identifiers = ['boy','boys','man','men']
girl_identifiers = ['girl','girls','woman','women','commitment']

In [6]:
def identify_gender(file_name_string):
    """ method to identify gender for given file name """
    for y in boys_identifiers:
        if y in file_name_string and "wo"+y not in file_name_string:
            return 'boy' 
    for x in girl_identifiers:
        if x in file_name_string:
            return 'girl' 

In [7]:
def populate_df_with_gender(input_df:object,input_string:str):
    """ populate dataframe with new column gender """
    output_df = input_df
    output_df['gender']=identify_gender(input_string)
    return output_df

#### Data Processors Methods

In [8]:
# columns names finalization

In [9]:
column_names_required = [
    'name',
    'high_school',
    'state',
    'commitment_year',
    'club',
    'club_year',
    'gender',
    'college',
    'position',
    'height',
    'width',
    'club_rank',
    ]
column_names_required

['name',
 'high_school',
 'state',
 'commitment_year',
 'club',
 'club_year',
 'gender',
 'college',
 'position',
 'height',
 'width',
 'club_rank']

In [10]:
def get_csv_dataframes_from_folder(folder_path:str):
    """ method to read multiple csv files inside single folder"""
    output_dfs_dict = {}
    for file in os.listdir(folder_path):
        output_dfs_dict[file]=pd.read_csv(os.path.join(folder_path,file))
    return output_dfs_dict

In [11]:
def map_col_names(
    input_df:object,
    col_names_map_dict:dict
    ):
    """ map scrapped csv col names to required column names list """
    return input_df.rename(columns=col_names_map_dict)

In [12]:
def merge_first_last(
    input_df:object,
    cols_to_merge = ['First','Last'],
    new_col_name = 'name'
    ):
    """ merge First Name, Last Name column into one column called name """
    output_df = input_df
    for col in cols_to_merge:
        output_df[col]=output_df[col].astype(str)
    output_df[new_col_name]= output_df[cols_to_merge].agg(' '.join, axis=1)
    output_df = output_df.drop(cols_to_merge,axis=1)
    return output_df

In [13]:
def set_col_names_lower(input_df:object):
    """ lower column names """
    return input_df.set_axis([x.lower() for x in list(input_df.columns)],axis=1)

In [14]:
def set_col_names_underscore_separated(input_df:object):
    """make column names single underscore separated"""
    return input_df.set_axis([re.sub(' +','_',x) for x in list(input_df.columns)],axis=1)

In [15]:
def drop_unnamed_columns(input_df):
    """ drop unnamed columns from dataframe """
    df_ = input_df
    df_ = df_.loc[:, ~df_.columns.str.contains('^unnamed')]
    return df_

In [16]:
def refactor_col_names(
    input_df:object,
    col_names_map_dict:dict,
    source_website='laxnumbers'
    ):
    """ refactor column names to keep consistency and follow naming convention (lowercase, underscore-separated """
    output_df = input_df
    if source_website == 'americanselectlacrosse':
        output_df = merge_first_last(
            output_df,
            ['First','Last'],
            'name'
        )
    elif source_website == 'toplaxrecruits':
        output_df = merge_first_last(
            output_df,
            ['First Name','Last Name'],
            'name'
        )
    output_df=map_col_names(output_df,col_names_map_dict)
    output_df=set_col_names_lower(output_df)
    output_df=set_col_names_underscore_separated(output_df)
    return output_df
    

#### populate commitment year column

In [17]:
# populate commitment year in dataframe

def get_year_from_string(file_name:str):
    """ detects year from file name string """
    try:
        output = re.match(r'.*([1-3][0-9]{3})', file_name).group(1)
        return output
    except:
        print('year not found')
        return None

In [18]:
def populate_commitment_year_from_filename(input_df,file_name:None):
    """ populate dataframe using  year from filename string """
    year = get_year_from_string(file_name)
    output_df = input_df
    output_df['commitment_year']=year
    return output_df

##### columns mapping dictionaries

In [19]:
american_select_names_map_dict = {
    'Region':'state',
    'School': 'college',
    'CLUB TEAM':'club',
    }

In [20]:
clublacrose_names_map_dict = {
    'Player Name':'name',
    'University':'college',
    'Division': 'commitment_year',
    }

In [21]:
laxnumbers_names_map_dict = {
    'Name':'name',
    'State': 'state',
    'Club': 'club',
    'Previous  School':'high_school',
    'Class':'commitment_year',
    'Pos':'position'
    }

In [22]:

toplax_names_map_dict = {
    'High School Name':'high_school',
    'Grad Year':'commitment_year',
    'Graduation Year':'commitment_year',
    'region':'state',
    'ST':'state',
    'COLLEGE COMMITTED TO': 'college',
    'College Committed To/Signed With':'college',
    'College Committed to':'college',
    'Club Team':'club',
    'DIV':'division'
    }


#### __main__

#### RAW TO BRONZE Layer

#### process data from raw to bronze layer

In [23]:
DATA_FOLDER_PATH = '../data/'
FOLDER_NAME = 'americanselectlacrosse'
WRITE_FLAG = True
COLS_TO_MERGE = ['First','Last']

In [24]:
def extract_and_process_data_from_raw_folder(
        raw_folder_path:str,
        column_names_map_dict:object
    ):
    """ read csv files and extract and transform data ready for bronze layer """
    csv_dfs_dict = get_csv_dataframes_from_folder(raw_folder_path)
    print('csv dfs read success')
    source_website_name = raw_folder_path.split('/')[-2]
    print(source_website_name)
    bronze_csv_dfs_dict = {}
    for raw_file_name, df in csv_dfs_dict.items():
        if "americanselect" in raw_file_name:
            # populate commitment year from file name
            df=populate_commitment_year_from_filename(df,raw_file_name)
            # merge first and last column names
            df = merge_first_last(df,['First','Last'],'name')
        if "toplax" in raw_file_name:
            df = merge_first_last(df,['First Name','Last Name'],'name')
        
        bronze_file_name = revamp_file_name(raw_file_name)
        # refactor column names
        df_refactored = refactor_col_names(df,column_names_map_dict,source_website_name)
        # populate dataframe with gender column
        df_refactored = populate_df_with_gender(df_refactored,raw_file_name)

        # populate missing columns from required list with None value default
        missing_columns = list(set(column_names_required)-set(list(df_refactored.columns)))
        df_refactored[missing_columns]=None
        df_refactored = drop_unnamed_columns(df_refactored)
        bronze_csv_dfs_dict[bronze_file_name]=df_refactored
        
    return bronze_csv_dfs_dict


In [25]:
def write_dfs_dict_to_folder(
    input_dict:dict,
    folder_path:str
    ):
    """ writes dataframes in dict into target folder """
    for file_name, df in input_dict.items():
        df.to_csv(os.path.join(folder_path,file_name))
        print(file_name+' written to '+folder_path+ ' success ... ')
    return None

In [26]:
def process_bronze_to_silver(
        bronze_folder_path:str,
        unique_columns = [],
    ):
    # read all csv files
    # concat all dataframe into one 
    # deal null values
    # dedupe based on unique columns
    dfs = []
    for root,dirs,files in os.walk(bronze_folder_path):
        for file in files:
            if file.endswith(".csv"):
                dfs.append(pd.read_csv(os.path.join(root,file)))
    final_df = pd.concat(dfs)
    # now dedupe based on unique columns

    return final_df

#### Workshop

In [29]:
import sys
sys.path.insert(2,'../etl/')
from data_helper import process_data_from_bronze_to_silver
from data_helper import process_data_from_sliver_to_gold
from data_helper import create_none_existing_folder

In [30]:
# process brozne to silver

bronze_folder_path = '../data/commitments/bronze'
silver_folder_path = '../data/commitments/silver'
gold_folder_path = '../data/commitments/gold'
create_none_existing_folder(silver_folder_path)
create_none_existing_folder(gold_folder_path)


../data/commitments/silver already exists
../data/commitments/gold already exists


In [31]:
process_data_from_bronze_to_silver(bronze_folder_path,silver_folder_path)

extracting from clublacrosse
saving into silver folder ../data/commitments/silver
success 
extracting from americanselectlacrosse
saving into silver folder ../data/commitments/silver
success 
extracting from laxnumbers
saving into silver folder ../data/commitments/silver
success 
extracting from toplaxrecruits
saving into silver folder ../data/commitments/silver
success 


In [32]:
process_data_from_sliver_to_gold(
    silver_folder_path=silver_folder_path,
    gold_folder_path=gold_folder_path,
    output_file_name='commitments.csv'
    )

extracting data from ../data/commitments/silver


/home/bilal/Desktop/lancing/lacrosse-prediction/notebooks/../etl/data_helper.py:255: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  dfs_dict = get_csv_dataframes_from_folder(silver_folder_path)


saving into ../data/commitments/gold
commitments.csv saved success
